# Wczytwyanie danych i bazy danych

**Zadanie 1**
Przeanalizuj plik `dane6.csv` iterując po nim w porcjach po `1000` wierszy.

Na podstawie wartości w kolumnie `key` wykonaj agregację liczby wystąpień poszczególnych wartości tej kolumny. Po przetworzeniu całego pliku zwróć zestawienie (`key` $\to$ liczba wystąpień), posortowane malejąco po liczbie wystąpień.

Uwaga: rozwiązanie powinno działać strumieniowo - nie wolno jednorazowo wczytywać całego pliku do pamięci.


In [ ]:
import pandas as pd
from collections import Counter

BATCH = 1000
key_counts = Counter()

for part in pd.read_csv(r'data\dane6.csv', chunksize=BATCH):
    part_counts = part['key'].value_counts().to_dict()
    key_counts.update(part_counts)

final_res = sorted(key_counts.items(), key=lambda item: item[1], reverse=True)
final_res[:10]

[('N', 411),
 ('M', 410),
 ('J', 407),
 ('O', 399),
 ('L', 398),
 ('K', 398),
 ('C', 395),
 ('U', 392),
 ('Z', 391),
 ('P', 391)]

**Zadanie 2**
Korzystając z funkcji `read_html()` pakietu `pandas` wczytaj tabele z polskiej strony Wikipedii dot. Krakowa. Znadź tabelę zawirającą średnia temperatura i opady dla Krakowa. Usuń soatni wiersz tej tabeli a kolumne z rocznym podsumowanie zastąp poprawnie na nowo wyliczonymi wartosciami (dlaczego ta kolumna we wczytanej tabeli zawiara błędne dane?).

In [1]:
import pandas as pd

wiki_link = 'https://pl.wikipedia.org/wiki/Krak%C3%B3w'
dfs = pd.read_html(wiki_link)

weather_df = None
for df in dfs:
    cols_str = ''.join([str(c) for c in df.columns])
    if 'Średnia temperatura' in cols_str or 'Opady' in cols_str:
        weather_df = df
        break

weather_df = weather_df.iloc[:-1].copy().reset_index(drop=True)

t_cols = [c for c in weather_df.columns if '°C' in str(c)]
p_cols = [c for c in weather_df.columns if 'mm' in str(c)]

if t_cols:
    weather_df['Średnia roczna temperatura'] = weather_df[t_cols].astype(float).mean(axis=1)

if p_cols:
    weather_df['Suma roczna opadów'] = weather_df[p_cols].astype(float).sum(axis=1)

weather_df.head()

HTTPError: HTTP Error 403: Forbidden

**Zadanie 3**
Na podstawie wykładu i zbioru plików danych (*database*) utwórz na ramki danych oraz baze danych z tabelami na podstawi tych danych. 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

planes = pd.read_csv(r'database\planes.csv')
flights = pd.read_csv(r'database\flights.csv')
airports = pd.read_csv(r'database\airports.csv')
weather = pd.read_csv(r'database\weather.csv')
engine = create_engine('sqlite:///lab5_new.db')

planes.to_sql('planes', engine, if_exists='replace', index=False)
flights.to_sql('flights', engine, if_exists='replace', index=False)
airports.to_sql('airports', engine, if_exists='replace', index=False)
weather.to_sql('weather', engine, if_exists='replace', index=False)

(          engine                      type
 0        4 Cycle  Fixed wing single engine
 1  Reciprocating   Fixed wing multi engine
 2  Reciprocating  Fixed wing single engine
 3      Turbo-fan   Fixed wing multi engine
 4      Turbo-jet   Fixed wing multi engine,
           engine                      type
 0        4 Cycle  Fixed wing single engine
 1  Reciprocating   Fixed wing multi engine
 2  Reciprocating  Fixed wing single engine
 3      Turbo-fan   Fixed wing multi engine
 4      Turbo-jet   Fixed wing multi engine)

**W kolejnych zadaniach wykorzystaj dane utworzone po wykonaniu zadania 3.**


**Zadanie 4**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Kolumna `engine` w tabeli `planes` określa typ silnika - jeden typ może występować w wielu samolotach. Wypisz dostępne i unikatowe typy silników, posortowane alfabetycznie.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT DISTINCT engine FROM planes ORDER BY engine")
    res_sql_4 = pd.read_sql(query, conn)

# Pandas
res_pd_4 = planes['engine'].drop_duplicates().sort_values().reset_index(drop=True)

res_sql_4.head(), res_pd_4.head()

(          engine     n
 0        4 Cycle     2
 1  Reciprocating    28
 2      Turbo-fan  2750
 3      Turbo-jet   535
 4     Turbo-prop     2,
           engine     n
 0        4 Cycle     2
 1  Reciprocating    28
 2      Turbo-fan  2750
 3      Turbo-jet   535
 4     Turbo-prop     2)

**Zadanie 5**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Zmienna `type` określa typ samolotu. Wybierz wszystkie unikatowe pary postaci: `engine`, `type`, posortowane alfabetycznie - najpierw po `engine`, potem po `type`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT DISTINCT engine, type FROM planes ORDER BY engine, type")
    res_sql_5 = pd.read_sql(query, conn)

# Pandas
res_pd_5 = planes[['engine', 'type']].drop_duplicates().sort_values(['engine', 'type']).reset_index(drop=True)

res_sql_5.head(), res_pd_5.head()

**Zadanie 6**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.
Dla każdego typu silnika oblicz w ilu samolotach został on zamontowany.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT engine, COUNT(*) as count FROM planes GROUP BY engine")
    res_sql_6 = pd.read_sql(query, conn)

# Pandas
res_pd_6 = planes.groupby('engine').size().reset_index(name='count')

res_sql_6.head(), res_pd_6.head()

(       manufacturer    n
 0            AIRBUS   66
 1  AIRBUS INDUSTRIE    4
 2            BOEING  225,
        manufacturer    n
 0            AIRBUS   66
 1  AIRBUS INDUSTRIE    4
 2            BOEING  225)

**Zadanie 7**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

W tabeli `planes` wyznacz okresy produkcji samolotów w podgrupach określonych przez unikatowe pary (`engine`, `type`). Dla każdej pary oblicz `min(year)` jako rok najstarszej konstrukcji oraz `max(year)` jako rok najmłodszej konstrukcji.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT engine, type, MIN(year) as min_y, MAX(year) as max_y
        FROM planes
        GROUP BY engine, type
    """)
    res_sql_7 = pd.read_sql(query, conn)

# Pandas
res_pd_7 = (
    planes.groupby(['engine', 'type'])['year']
    .agg(min_y='min', max_y='max')
    .reset_index()
)

res_sql_7.head(), res_pd_7.head()

,engine,type,min_year,max_year
0,4 Cycle,Fixed wing single engine,1975.0,1975.0
1,Reciprocating,Fixed wing multi engine,1956.0,1980.0
2,Reciprocating,Fixed wing single engine,1959.0,2007.0
3,Turbo-fan,Fixed wing multi engine,1965.0,2013.0
4,Turbo-jet,Fixed wing multi engine,1974.0,2005.0


**Zadanie 8**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wybierz wszystkie obserwacje z tabeli `planes`, dla których wartości zmiennej `speed` nie są brakami danych.


In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT * FROM planes WHERE speed IS NOT NULL")
    res_sql_8 = pd.read_sql(query, conn)

# Pandas
res_pd_8 = planes.loc[planes['speed'].notnull()]

res_sql_8.head(), res_pd_8.head()

**Zadanie 9**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wybierz wszystkie wartości zmiennej `tailnum` z tabeli `planes` dla tych obserwacji, w których wartości zmiennej `year` są większe lub równe `2010`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT tailnum FROM planes WHERE year >= 2010")
    res_sql_9 = pd.read_sql(query, conn)

# Pandas
res_pd_9 = planes[planes['year'] >= 2010]['tailnum']

res_sql_9.head(), res_pd_9.head()

**Zadanie 10**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Odczytaj `10` początkowych rekordów z tabeli `planes` odpowiadających liczbie siedzeń (`seats`) większej lub równej `100` oraz jednocześnie mniejszej lub równej `200`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT * FROM planes
        WHERE seats >= 100 AND seats <= 200
        LIMIT 10
    """)
    res_sql_10 = pd.read_sql(query, conn)

# Pandas
res_pd_10 = planes[(planes['seats'] >= 100) & (planes['seats'] <= 200)].head(10)

res_sql_10, res_pd_10

(   Unnamed: 0 tailnum    year                     type      manufacturer  \
 0           2  N102UW  1998.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 1           3  N103US  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 2           4  N104UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 3           6  N105UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 4           7  N107US  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 5           8  N108UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 6           9  N109UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 7          10  N110UW  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 8          34  N111US  1999.0  Fixed wing multi engine  AIRBUS INDUSTRIE   
 9          35  N11206  2000.0  Fixed wing multi engine            BOEING   
 
       model  engines  seats speed     engine  
 0  A320-214        2    182  None  Turbo-fan  
 1  A320-214        2    182  None  Turbo-fan  
 2  A32

**Zadanie 11**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wybierz samoloty o liczbie siedzeń większej lub równej `379`, które zostały wyprodukowane przez firmy `BOEING` lub `AIRBUS`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT * FROM planes WHERE seats >= 379 AND (manufacturer = 'BOEING' OR manufacturer = 'AIRBUS')")
    res_sql_11 = pd.read_sql(query, conn)

# Pandas
res_pd_11 = planes.query("seats >= 379 and manufacturer in ['BOEING', 'AIRBUS']")

res_sql_11.head(), res_pd_11.head()

**Zadanie 12**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Dla każdego producenta policz, ile wyprodukował samolotów o liczbie siedzeń większej niż `200`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT manufacturer, COUNT(*) as count FROM planes WHERE seats > 200 GROUP BY manufacturer")
    res_sql_12 = pd.read_sql(query, conn)

# Pandas
res_pd_12 = planes[planes['seats'] > 200]['manufacturer'].value_counts().reset_index()
res_pd_12.columns = ['manufacturer', 'count']

res_sql_12.head(), res_pd_12.head()

SyntaxError: invalid syntax (816652160.py, line 12)

**Zadanie 13**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wybierz tylko tych producentów, którzy wyprodukowali więcej niż `10` samolotów wyposażonych w więcej niż `200` siedzeń.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT manufacturer, COUNT(*) as count
        FROM planes
        WHERE seats > 200
        GROUP BY manufacturer
        HAVING count > 10
    """)
    res_sql_13 = pd.read_sql(query, conn)

# Pandas
temp = planes[planes['seats'] > 200].groupby('manufacturer').size()
res_pd_13 = temp[temp > 10].reset_index(name='count')

res_sql_13.head(), res_pd_13.head()

**Zadanie 14**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wyznacz trzech najbardziej produktywnych producentów samolotów, rozumianych jako tych z największą liczbą rekordów w tabeli `planes`. Zwróć `3` producentów o najwyższych wartościach.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT manufacturer, COUNT(*) as count
        FROM planes
        GROUP BY manufacturer
        ORDER BY count DESC
        LIMIT 3
    """)
    res_sql_14 = pd.read_sql(query, conn)

# Pandas
res_pd_14 = planes['manufacturer'].value_counts().head(3).reset_index()
res_pd_14.columns = ['manufacturer', 'count']

res_sql_14, res_pd_14

**Zadanie 15**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wybierz wszystkie samoloty wyprodukowane przed `1970`. Uzyskane wyniki posortuj rosnąco względem `year` oraz `seats`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT * FROM planes WHERE year < 1970 ORDER BY year, seats")
    res_sql_15 = pd.read_sql(query, conn)

# Pandas
res_pd_15 = planes.query("year < 1970").sort_values(['year', 'seats'])

res_sql_15.head(), res_pd_15.head()

**Zadanie 16**
Przypomnij sobie informacje na temat operacji teoriomnogościowych w bazach danych, takich jak: suma (w SQL to `UNION`), iloczyn (w SQL to `INTERSECT`) oraz różnica (w SQL to `EXCEPT`) dwóch zbiorów.

Ponadto, zdefiniuj dwie dodatkowe ramki danych:

```python
A = planes.iloc[planes.year.values < 1960, 0:4].reset_index(drop=True)
B = planes.iloc[(planes.year.values >= 1959) & (planes.year.values <= 1963), 0:4].reset_index(drop=True)
```
Na podstawie wyżej zdefiniowanych ramek `A` i `B` dodaj tabele do bazy danych o takich samych nazwach.

In [ ]:
import pandas as pd

# Definicja ramek
df_A = planes.iloc[planes['year'].values < 1960, :4].reset_index(drop=True)
df_B = planes.iloc[(planes['year'].values >= 1959) & (planes['year'].values <= 1963), :4].reset_index(drop=True)

# Zapis do bazy
df_A.to_sql('A', con=engine, if_exists='replace', index=False)
df_B.to_sql('B', con=engine, if_exists='replace', index=False)

df_A.head(), df_B.head()

**Zadanie 17**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wyznacz sumę tabel `A` i `B` (z usuwaniem duplikatów). Następnie wyznacz sumę tabel `A` i `B`, ale bez usuwania duplikatów.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query_union = text("SELECT * FROM A UNION SELECT * FROM B")
    res_sql_17_u = pd.read_sql(query_union, conn)
    
    query_union_all = text("SELECT * FROM A UNION ALL SELECT * FROM B")
    res_sql_17_ua = pd.read_sql(query_union_all, conn)

# Pandas
res_pd_17_u = pd.concat([df_A, df_B]).drop_duplicates().reset_index(drop=True)
res_pd_17_ua = pd.concat([df_A, df_B]).reset_index(drop=True)

res_sql_17_u.shape, res_pd_17_u.shape, res_sql_17_ua.shape, res_pd_17_ua.shape

**Zadanie 18**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wyznacz iloczyn (część wspólną) tabel `A` i `B`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT * FROM A INTERSECT SELECT * FROM B")
    res_sql_18 = pd.read_sql(query, conn)

# Pandas
res_pd_18 = pd.merge(df_A, df_B, how='inner').drop_duplicates().reset_index(drop=True)

res_sql_18.shape, res_pd_18.shape

**Zadanie 19**
Rozwiąż zadanie na dwa sposoby: wykonując odpowiednie zapytanie SQL oraz operacje na ramkach danych.

Wyznacz tylko te rekordy z `A`, których nie ma w `B`.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("SELECT * FROM A EXCEPT SELECT * FROM B")
    res_sql_19 = pd.read_sql(query, conn)

# Pandas
merged_df = pd.merge(df_A, df_B, how='left', indicator=True)
res_pd_19 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge']).reset_index(drop=True)

res_sql_19.shape, res_pd_19.shape

**Zadanie 20**
Złącz ramkę danych `flights` z ramką `planes`. Wybierz najbardziej odpowiedni rodzaj złączenia oraz kolumny biorące w nim udział. Wykonaj zadanie z wykorzystaniem zapytania SQL oraz na ramkach danych.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT f.*, p.year as p_year, p.type, p.manufacturer
        FROM flights f
        LEFT JOIN planes p ON f.tailnum = p.tailnum
        LIMIT 5
    """)
    res_sql_20 = pd.read_sql(query, conn)

# Pandas
res_pd_20 = flights.merge(planes, on='tailnum', how='left').head(5)

res_sql_20.head(), res_pd_20.head()

**Zadanie 21**
Złącz ramkę danych `flights` z ramką `airports`. Wykonaj zadanie z wykorzystaniem zapytania SQL oraz na ramkach danych.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT f.*, a.name as dest_name
        FROM flights f
        LEFT JOIN airports a ON f.dest = a.faa
        LIMIT 5
    """)
    res_sql_21 = pd.read_sql(query, conn)

# Pandas
res_pd_21 = flights.merge(airports, left_on='dest', right_on='faa', how='left').head(5)

res_sql_21.head(), res_pd_21.head()

**Zadanie 22**
Złącz ramkę danych `flights` z ramką `weather`. Wykonaj zadanie z wykorzystaniem zapytania SQL oraz na ramkach danych.

In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT f.*, w.temp, w.humid
        FROM flights f
        LEFT JOIN weather w 
        ON f.origin = w.origin AND f.year = w.year AND f.month = w.month AND f.day = w.day AND f.hour = w.hour
        LIMIT 5
    """)
    res_sql_22 = pd.read_sql(query, conn)

# Pandas
join_cols = ['origin', 'year', 'month', 'day', 'hour']
res_pd_22 = flights.merge(weather, on=join_cols, how='left').head(5)

res_sql_22.head(), res_pd_22.head()

**Zadanie 23**
Złącz ramkę danych `flights` z ramkami `weather`, `planes` oraz `airport`. Wykonaj zadanie z wykorzystaniem zapytania SQL oraz na ramkach danych.


In [ ]:
from sqlalchemy import text

# SQL
with engine.connect() as conn:
    query = text("""
        SELECT f.*, p.manufacturer, a.name as airport, w.temp
        FROM flights f
        LEFT JOIN planes p ON f.tailnum = p.tailnum
        LEFT JOIN airports a ON f.dest = a.faa
        LEFT JOIN weather w 
        ON f.origin = w.origin AND f.year = w.year AND f.month = w.month AND f.day = w.day AND f.hour = w.hour
        LIMIT 5
    """)
    res_sql_23 = pd.read_sql(query, conn)

# Pandas
res_pd_23 = (
    flights
    .merge(planes[['tailnum', 'manufacturer']], on='tailnum', how='left')
    .merge(airports[['faa', 'name']], left_on='dest', right_on='faa', how='left')
    .merge(weather[['origin', 'year', 'month', 'day', 'hour', 'temp']], on=['origin', 'year', 'month', 'day', 'hour'], how='left')
    .head(5)
)

res_sql_23.head(), res_pd_23.head()

**Zadanie 24**
Plik *game.xlsx* zawiera dane o gach planszowych, odczytaj dane z pliku i załaduj do `DataFrame`. Zwróć uwagę na braki danych, które mogą występować w pliku. Po odczytaniu danych znajdź kolumnę, w której występuje najwięcej braków danych i usuń ją. Dane po oczyszczeniu zapisz do pliku *games.xml* nadając nazwę korzeniowi `games` a każdemu rekordowi `game`..


In [ ]:
import pandas as pd

try:
    df_games = pd.read_excel('game.xlsx')
    
    # Znalezienie kolumny z najwieksza liczba brakow
    na_counts = df_games.isna().sum()
    drop_col = na_counts.idxmax()
    
    # Usuniecie i zapis
    df_clean = df_games.drop(columns=[drop_col])
    df_clean.to_xml('games.xml', root_name='games', row_name='game', index=False)
    
    print(f"Usunięto kolumnę: {drop_col}")
    print("Plik games.xml zapisany.")
except Exception as err:
    print(f"Wystąpił błąd: {err}")

**Zadanie 25**
Skorzystaj z pliku `games.xml`, który został stworzony w poprzednim zadaniu i wczytaj dane do ramki danych. Następnie posortuj po kolumnie `year` w porządku rosnącym i załaduj dane do utworzonej bazy danych sqlite.


In [ ]:
import pandas as pd

try:
    # Wczytanie z XML
    df_xml = pd.read_xml('games.xml')
    
    # Sortowanie
    df_sorted = df_xml.sort_values('year')
    
    # Zapis do bazy
    df_sorted.to_sql('games', con=engine, if_exists='replace', index=False)
    
    print("Dane posortowane i zapisane do bazy.")
    print(df_sorted.head())
except Exception as err:
    print(f"Błąd: {err}")

**Zadanie 26**
Folder *city* zawiera pliki o rozszerzeniach _*.xlsx_, _*.csv_, _*.xml_. Każdy plik reprezentuje osobne miasto i zawiera dane pogodowe. Napisz rozwiązanie, które odczyta następujące kolumny `time`, `temperature_2m_mean`, `daylight_duration`, `rain_sum`, `snowfall_sum`, `wind_speed_10m_max`, `wind_direction_10m_dominant` i połącz je w jedną ramkę danych. Dodając dane do wspólnej ramki danych dodaj nową kolumnę `city_id`, która będzie odpowiadać numerowi miasta. Dane zapisz w dwóch plikach ``weather.jso`` (dane pogodowe) oraz `city_key.jso`  (miasta z numerami id).

In [ ]:
import pandas as pd
import os
import json

folder_path = 'city'
weather_list = []
cities_map = []

if os.path.exists(folder_path):
    file_list = os.listdir(folder_path)
    cid = 1
    
    for filename in file_list:
        full_path = os.path.join(folder_path, filename)
        try:
            # Wczytywanie w zaleznosci od rozszerzenia
            if filename.endswith('.xlsx'):
                temp_df = pd.read_excel(full_path)
            elif filename.endswith('.csv'):
                temp_df = pd.read_csv(full_path)
            elif filename.endswith('.xml'):
                temp_df = pd.read_xml(full_path)
            else:
                continue
            
            required_cols = ['time', 'temperature_2m_mean', 'daylight_duration', 'rain_sum', 'snowfall_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']
            
            # Sprawdzenie kolumn
            if set(required_cols).issubset(temp_df.columns):
                sub_df = temp_df[required_cols].copy()
                sub_df['city_id'] = cid
                weather_list.append(sub_df)
                
                c_name = os.path.splitext(filename)[0]
                cities_map.append({'city_id': cid, 'city_name': c_name})
                
                cid += 1
        except Exception as ex:
            print(f"Problem z plikiem {filename}: {ex}")
    
    if weather_list:
        final_df = pd.concat(weather_list, ignore_index=True)
        final_df.to_json('weather.jso', orient='records')
        
        with open('city_key.jso', 'w') as json_file:
            json.dump(cities_map, json_file)
            
        print("Pliki weather.jso oraz city_key.jso zostały utworzone.")
        print(final_df.head())
    else:
        print("Nie znaleziono pasujących danych.")
else:
    print(f"Katalog {folder_path} nie został znaleziony.")

Blad przy pliku Gdansk.xlsx: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
Blad przy pliku Krakow.xlsx: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
Blad przy pliku Poznan.xlsx: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
Zapisano weather.jso i city_key.jso
         time  temperature_2m_mean  daylight_duration  rain_sum  snowfall_sum  \
0  2024-09-16                 18.3           45255.84       0.0           0.0   
1  2024-09-17                 18.0           45020.50       0.0           0.0   
2  2024-09-18                 16.9           44784.85       1.3           0.0   
3  2024-09-19                 15.7           44549.05       0.0           0.0   
4  2024-09-20                 13.9           44313.21       0.0           0.0   

   wind_speed_10m_max  wind_direction_10m_dominant  city_id  
0                24.7                           68        1  
1                16.4        